<a href="https://www.kaggle.com/code/aleksandrmorozov123/recommender-systems?scriptVersionId=188133798" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv


**Content-based recommender systems**

In [2]:
# import required libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from gensim import models
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
%matplotlib inline
from gensim.models import FastText as ft
from IPython.display import Image
import os

# import the data
Content_df = pd.read_csv("/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")

#Viewing Top 5 Rows
Content_df.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
# Data Info
Content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [4]:
Content_df.shape

(23486, 11)

In [5]:
# convert to strings
unique_df = Content_df['Review Text'].astype(str)

# Converting String to Lower Case
unique_df = unique_df.apply(lambda x: x.lower())

# Remove Stop special Characters
unique_df = unique_df.apply(lambda x: re.sub(r'[^\w\s]', '', x))
# Coverting Description to List
desc_list = list(unique_df)
unique_df= unique_df.reset_index(drop=True)

#Viewing Top 5 Rows
unique_df.head(5)

0    absolutely wonderful  silky and sexy and comfo...
1    love this dress  its sooo pretty  i happened t...
2    i had such high hopes for this dress and reall...
3    i love love love this jumpsuit its fun flirty ...
4    this shirt is very flattering to all due to th...
Name: Review Text, dtype: object

**Similarity measures**

In [6]:
# import Count Vectorizer from Scikit-Learn
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer(stop_words='english')

# Euclidean distance
def find_euclidean_distances(sim_matrix, index, n=10):
    # Getting Score and Index
    result = list(enumerate(sim_matrix[index]))
    # Sorting the Score and taking top 10 products
    sorted_result = sorted(result,key=lambda x:x[1],reverse=False)[1:10+1]
    # Mapping index with data
    similar_products = [{'value': unique_df.iloc[x[0]],
                         'score' : round(x[1], 2)} for x in sorted_result]
    return similar_products

#Cosine similarity
def find_similarity(cosine_sim_matrix, index, n=10):
    # calculate cosine similarity between each vectors
    result = list(enumerate(cosine_sim_matrix[index]))
    # Sorting the Score
    sorted_result = sorted(result,key=lambda x:x[1],reverse=True)[1:n+1]
    similar_products = [{'value': unique_df.iloc[x[0:]],
                         'score' : round(x[1], 2)} for x in sorted_result]
    return similar_products

#Manhattan distance
def find_manhattan_distance(sim_matrix, index, n=10):
    # Getting Score and Index
    result = list(enumerate(sim_matrix[index]))
    # Sorting the Score and taking top 10 products
    sorted_result = sorted(result,key=lambda x:x[1],reverse=False)[1:10+1]
    # Mapping index with data
    similar_products = [{'value': unique_df.iloc[x[0:]],
                         'score' : round(x[1], 2)} for x in sorted_result]
    return similar_products

#Comparing similarity to get the top matches using count Vec
def get_recommendation_cv(product_id, df, similarity, n=10):
    row = unique_df.loc[df == product_id]
    index = list(row.index)[0:]
    description = row.loc[index]
    #Create vector using Count Vectorizer
    count_vector = cnt_vec.fit_transform(desc_list)
    if similarity == "cosine":
        sim_matrix = cosine_similarity(count_vector)
        products = find_similarity(sim_matrix , index)
    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(count_vector)
        products = find_manhattan_distance(sim_matrix, index)
    else:
        sim_matrix = euclidean_distances(count_vector)
        products = find_euclidean_distances(sim_matrix, index)
    return products